In [2]:
### Import Packages ###
import os
import ast
import numpy as np
import math as math
import pandas as pd
import random as random
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist

### Import functions ###
from utils.Main import *
from utils.Selector import *
from utils.Auxiliary import *
from utils.Prediction import *

### Get Directory ###
cwd = os.getcwd()
ParentDirectory = os.path.abspath(os.path.join(cwd, ".."))

### Parameter Vector ###
ParameterVector = pd.read_csv(os.path.join(ParentDirectory, "Data", "raw", "ParameterVectorSimulations.csv"))

# Simulation

In [3]:

JobName = "Seed0_DataSimulate_TP0.2_CP0.8_STGSx_MTLinearRegression_N1000_K5_alpha_val0.1"
Seed =  "0"
Data =  "Simulate"
TestProportion =  "0.2"
CandidateProportion =  "0.8"
SelectorType =  "iGSFunction"
ModelType =  "LinearRegressionFunction"
DataArgs =  '{"N": 1000, "K": 5}'
SelectorArgs =  '{"df_Candidate": "df_Candidate"}'
ModelArgs =  '{"alpha_val": 0.1}'
Output =  "/TESTRESULTS.csv"


In [ ]:
### Set Up ###
ErrorVecSimulation = []
HistoryVecSimulation = []

### Run Code ###
for i in range(0,100):

    ### Set Up ###
    print(i)
    DataFileInput = "BostonHousing"
    # ModelType =  RandomForestRegressorFunction
    # ModelArgs = {"df_Train" : "df_Train",
    #              "n_estimators" : 50,
    #              "Seed" : 1}

    ### Sequential Learning Process ###
    ErrorVec, HistoryVec, SimulationParameters = OneIterationFunction(DataFileInput = DataFileInput,
                                                    Seed = ParameterVector.iloc[i]["Seed"],
                                                    TestProportion = ParameterVector.iloc[i]["TestProportion"],
                                                    CandidateProportion = ParameterVector.iloc[i]["CandidateProportion"],
                                                    SelectorType = globals().get(ParameterVector.iloc[i]["SelectorType"], None), 
                                                    ModelType = globals().get(ParameterVector.iloc[i]["ModelType"], None), 
                                                    DataArgs = ast.literal_eval(ParameterVector.iloc[i]["DataArgs"]),
                                                    SelectorArgs = ast.literal_eval(ParameterVector.iloc[i]["SelectorArgs"].replace("[","").replace("]","")),
                                                    ModelArgs = ast.literal_eval(ParameterVector.iloc[i]["ModelArgs"].replace("[","").replace("]","")))

    ErrorVecSimulation.append(ErrorVec)
    HistoryVecSimulation.append(HistoryVec)

    ErrorVec.to_csv(os.path.join(ParentDirectory, 
             "Results",
             str(DataFileInput).replace("Function", ""),
             "RandomForest",
             ) + str(ParameterVector.iloc[i]["Output"]))

# Applied

In [ ]:
### Set Up ###
ErrorVecSimulation = []
HistoryVecSimulation = []

### Run Code ###
for i in range(0,100):

    ### Set Up ###
    print(i)
    DataFileInput = "BostonHousing"

    ### Sequential Learning Process ###
    ErrorVecPassive, HistoryVecPassive, SimulationParametersPassive = OneIterationFunction(DataFileInput = DataFileInput,
                                                    Seed = i,
                                                    TestProportion = 0.2,
                                                    CandidateProportion = 0.8,
                                                    SelectorType = PassiveLearning, 
                                                    ModelType = RandomForestRegressorFunction, 
                                                    DataArgs = {},
                                                    SelectorArgs = {"df_Train" : "df_Train"},
                                                    ModelArgs = {"df_Train" : "df_Train",
                                                                 "n_estimators" : 100,
                                                                 "Seed" : 1})
    
    ErrorVecGSX, HistoryVecGSX, SimulationParametersGSX = OneIterationFunction(DataFileInput = DataFileInput,
                                                    Seed = i,
                                                    TestProportion = 0.2,
                                                    CandidateProportion = 0.8,
                                                    SelectorType = GSxFunction, 
                                                    ModelType = RandomForestRegressorFunction, 
                                                    DataArgs = {},
                                                    SelectorArgs = {"df_Train" : "df_Train"},
                                                    ModelArgs = {"df_Train" : "df_Train",
                                                                 "n_estimators" : 100,
                                                                 "Seed" : 1})
    
    ErrorVecGSy, HistoryVecGSy, SimulationParametersGSy = OneIterationFunction(DataFileInput = DataFileInput,
                                                    Seed = i,
                                                    TestProportion = 0.2,
                                                    CandidateProportion = 0.8,
                                                    SelectorType = GSyFunction, 
                                                    ModelType = RandomForestRegressorFunction, 
                                                    DataArgs = {},
                                                    SelectorArgs = {"df_Train" : "df_Train"},
                                                    ModelArgs = {"df_Train" : "df_Train",
                                                                 "n_estimators" : 100,
                                                                 "Seed" : 1})
    
    ErrorVeciGS, HistoryVeciGS, SimulationParametersiGS = OneIterationFunction(DataFileInput = DataFileInput,
                                                    Seed = i,
                                                    TestProportion = 0.2,
                                                    CandidateProportion = 0.8,
                                                    SelectorType = iGSFunction, 
                                                    ModelType = RandomForestRegressorFunction, 
                                                    DataArgs = {},
                                                    SelectorArgs = {"df_Train" : "df_Train"},
                                                    ModelArgs = {"df_Train" : "df_Train",
                                                                 "n_estimators" : 100,
                                                                 "Seed" : 1})

    ErrorVecSimulation.append(ErrorVec)
    HistoryVecSimulation.append(HistoryVec)

    ErrorVec.to_csv(os.path.join(ParentDirectory, 
             "Results",
             str(DataFileInput).replace("Function", ""),
             "RandomForest",
             ) + str(ParameterVector.iloc[i]["Output"]))

0


KeyboardInterrupt: 